# Introduction
A very important aspect of supervised and semi-supervised machine learning is the quality of the labels produced by human labelers. Unfortunately, humans are not perfect and in some cases may even maliciously label things incorrectly. In this assignment, you will evaluate the impact of incorrect labels on a number of different classifiers.

We have provided a number of code snippets you can use during this assignment. Feel free to modify them or replace them.


## Dataset
The dataset you will be using is the [Adult Income dataset](https://archive.ics.uci.edu/ml/datasets/Adult). This dataset was created by Ronny Kohavi and Barry Becker and was used to predict whether a person's income is more/less than 50k USD based on census data.

### Data preprocessing
Start by loading and preprocessing the data. Remove NaN values, convert strings to categorical variables and encode the target variable (the string <=50K, >50K in column index 14).

In [100]:
import pandas as pd
import numpy as np

In [101]:
# This can be used to load the dataset
# I changed header=None to header=0, so that it automatically assigns the column names
data = pd.read_csv("adult.csv", header=0, na_values='?')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [102]:
# Remove any rows containing NaN values
data = data.dropna()


In [103]:
# Convert categorical columns to category datatype
categorical_features = ['workclass', 'education', 'marital-status', 'occupation',
                      'relationship', 'race', 'sex', 'native-country', 'salary']

for column in categorical_columns:
    data[column] = data[column].astype('category')
    
# check if everything has worked out correctly
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             45222 non-null  int64   
 1   workclass       45222 non-null  category
 2   fnlwgt          45222 non-null  int64   
 3   education       45222 non-null  category
 4   education-num   45222 non-null  int64   
 5   marital-status  45222 non-null  category
 6   occupation      45222 non-null  category
 7   relationship    45222 non-null  category
 8   race            45222 non-null  category
 9   sex             45222 non-null  category
 10  capital-gain    45222 non-null  int64   
 11  capital-loss    45222 non-null  int64   
 12  hours-per-week  45222 non-null  int64   
 13  native-country  45222 non-null  category
 14  salary          45222 non-null  category
dtypes: category(9), int64(6)
memory usage: 2.8 MB


In [104]:
# encoding the target variable
salary_mapping = {'<=50K': 0, '>50K': 1}
data['salary'] = data['salary'].map(salary_mapping)

missing_salaries = data['salary'].isna().sum()

if missing_salaries > 0:
    print(f"There are {missing_salaries} missing values in the 'salary' column after mapping.")
    # drop the rows with missing salary values
    data = data.dropna(subset=['salary'])

# verify change
data.head(10)

There are 15060 missing values in the 'salary' column after mapping.


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0.0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0.0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0.0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0.0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0.0
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,0.0
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,0.0
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,1.0
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,1.0
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,1.0


### Data classification
Choose at least 4 different classifiers and evaluate their performance in predicting the target variable. 

#### Preprocessing
Think about how you are going to encode the categorical variables, normalization, whether you want to use all of the features, feature dimensionality reduction, etc. Justify your choices 

A good method to apply preprocessing steps is using a Pipeline. Read more about this [here](https://machinelearningmastery.com/columntransformer-for-numerical-and-categorical-data/) and [here](https://medium.com/vickdata/a-simple-guide-to-scikit-learn-pipelines-4ac0d974bdcf). 

<!-- #### Data visualization
Calculate the correlation between different features, including the target variable. Visualize the correlations in a heatmap. A good example of how to do this can be found [here](https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec). 

Select a features you think will be an important predictor of the target variable and one which is not important. Explain your answers. -->

#### Evaluation
Use a validation technique from the previous lecture to evaluate the performance of the model. Explain and justify which metrics you used to compare the different models. 

In [109]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Split data into features and target before building the pipeline as in the medium article
X = data.drop(columns='salary')
y = data['salary']

# Create transformers for different variable types (categorical and numerical in our case)
numerical_transformer = MinMaxScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

numerical_features = data.drop(columns='salary').select_dtypes(include=[np.number]).columns
categorical_features = data.drop(columns='salary').select_dtypes(include=['category']).columns

# Define your preprocessing steps here
steps = [('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)]

# Combine steps into a ColumnTransformer
ct = ColumnTransformer(steps)

# show the correlation between different features including target variable
def visualize(data, ct):
    # Apply the ColumnTransformer to the data (we need to remove the target first)
    X_transformed = ct.fit_transform(data.drop(columns='salary'))
    
    # Since ct transforms categorical features into multiple columns (one-hot encoded),
    # the transformed data will be a numpy array, so let's convert it to a DataFrame.
    transformed_columns = list(ct.get_feature_names_out())
    X_transformed_df = pd.DataFrame(X_transformed, columns=transformed_columns)
    
    # Add the target variable back to the transformed dataset
    X_transformed_df['salary'] = data['salary'].values
    
    # Compute the correlation matrix
    corr_matrix = X_transformed_df.corr()
    
    # Plot the correlation heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', square=True)
    plt.title('Correlation Matrix including Target Variable')
    plt.show()

# Apply your model to feature array X and labels y
def apply_model(model, X, y):    
    # Wrap the model and steps into a Pipeline
    pipeline = Pipeline(steps=[('t', ct), ('m', model)])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    pipeline.fit(X_train, y_train)
    
    # Evaluate the model and store results
    return evaluate_model(X_test, y_test, pipeline)

# Apply your validation techniques and calculate metrics
def evaluate_model(X_test, y_test, pipeline):
    # Make predictions on the test data
    y_pred = pipeline.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Return the evaluation metrics
    return {
        'accuracy': accuracy
    }

### Performing the classification using 4 random classifiers
#### (may take some time to perform this step)

In [110]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

#classifiers = {
#    "Logistic Regression": LogisticRegression(max_iter=500),
#    "Random Forest": RandomForestClassifier(),
#    "Decision Tree": DecisionTreeClassifier(),
#    "Support Vector Classifier": SVC()
#}

#for name, model in classifiers.items():
#    results = apply_model(model, X, y)
#    print(f"{name} Accuracy: {results['accuracy']:.2f}")

### Label perturbation
To evaluate the impact of faulty labels in a dataset, we will introduce some errors in the labels of our data.


#### Preparation
Start by creating a method which alters a dataset by selecting a percentage of rows randomly and swaps labels from a 0->1 and 1->0. 


In [116]:
def perturbate(y, fraction):
    y_array = y.to_numpy()  # Convert to numpy array

    # Determine the number of labels to flip
    n_flips = int(len(y_array) * fraction)

    # Randomly select indices to flip
    indices_to_flip = np.random.choice(len(y_array), size=n_flips, replace=False)

    # Flip the labels (assuming binary classification with labels 0 and 1)
    y_array[indices_to_flip] = 1 - y_array[indices_to_flip]

    return y_array

#### Analysis
Create a number of new datasets with perturbed labels, for fractions ranging from `0` to `0.5` in increments of `0.1`.

Perform the same experiment you did before, which compared the performances of different models except with the new datasets. Repeat your experiment at least 5x for each model and perturbation level and calculate the mean and variance of the scores. Visualize the change in score for different perturbation levels for all of the models in a single plot. 

State your observations. Is there a change in the performance of the models? Are there some classifiers which are impacted more/less than other classifiers and why is this the case?

#### Create Perturbed Datasets

In [126]:
def create_perturbed_datasets(y, increment=0.1, max_fraction=0.5):
    perturbed_datasets = {}
    fractions = np.arange(0, max_fraction + increment, increment)
    
    for fraction in fractions:
        perturbed_y = perturbate(y, fraction)
        perturbed_datasets[fraction] = perturbed_y
        
    return perturbed_datasets

perturbed_datasets = create_perturbed_datasets(y)
len(perturbed_datasets)

6

#### Evaluate Models (takes some time)

In [131]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=500),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier()
}

def evaluate_models_on_perturbed_data(models, perturbed_datasets, X, y, n_runs=1):
    results = {}
    
    for fraction, perturbed_y in perturbed_datasets.items():
        results[fraction] = {}
        
        for model_name, model in models.items():
            scores = []
            
            for _ in range(n_runs):
                score = apply_model(model, X, perturbed_y)
                scores.append(score['accuracy'])
                
            results[fraction][model_name] = {
                'mean_accuracy': np.mean(scores),
                'variance': np.var(scores)
            }
    
    return results

results = evaluate_models_on_perturbed_data(models, perturbed_datasets, X, y)

#### Visualizing results

In [132]:
def plot_results(results):
    plt.figure(figsize=(12, 6))
    
    for model_name in models.keys():
        means = []
        for fraction in sorted(result.keys()):
            means.append(results[fraction][model_name]['mean_accuracy'])
        
        plt.plot(sorted(results.keys()), means, marker='o', label=model_name)
        
    plt.title('Model Performance with Perturbed Labels')
    plt.xlabel('Fraction of Perturbed Labels')
    plt.ylabel('Mean Accuracy')
    plt.xticks(sorted(results.keys()))
    plt.legend()
    plt.grid()
    plt.show()
    
plot_results(results)

NameError: name 'result' is not defined

<Figure size 1200x600 with 0 Axes>

Observations + explanations: max. 400 words

#### Discussion

1)  Discuss how you could reduce the impact of wrongly labeled data or correct wrong labels. <br />
    max. 400 words



    Authors: Youri Arkesteijn, Tim van der Horst and Kevin Chong.


## Machine Learning Workflow

From part 1, you will have gone through the entire machine learning workflow which are they following steps:

1) Data Loading
2) Data Pre-processing
3) Machine Learning Model Training
4) Machine Learning Model Testing

You can see these tasks are very sequential, and need to be done in a serial fashion. 

As a small perturbation in the actions performed in each of the steps may have a detrimental knock-on effect in the task that comes afterwards.

In the final part of Part 1, you will have experienced the effects of performing perturbations to the machine learning model training aspect and the reaction of the machine learning model testing section.

## Part 2 Data Discovery

Now we have a some datasets that are related to each other.

**Altogether they are the same as the adult dataset used in the part 1 of the assignment.**

In this scenario, one can see the utility of the subsets of data can impact the outcome of the task from the previous section.

Because the data is split up, we want to be able to re-construct the data through data discovery.

As data discovery will allow you to be able to find relations that can be used to reconstruct the entire dataset.

Implement a method of your choice to perform the data discovery to be able to recover the entire dataset from part 1 of the assignment.

In [ ]:
def discovery_algorithm():
    """Function should be able to perform data discovery to find related datasets
    Possible Input: List of datasets
    Output: List of pairs of related datasets
    """

    pass

In [ ]:
## Do some testing using different degrees of relations on the downstream task.
##



### Evaluation

Following the same workflow as Part 1 of the assignment, you will need to perform the steps once again.
This means that with the difference in setting, there has now been a change in the data loading portion of the workflow. 

<!-- While performing data discovery, one can check how having more data of different relations can effect the outcome of the downstream task. -->

As you perform the act of data discovery you will be piecing the data back together one by one by finding the relation between the datasets.

As you piece the data back together, the entire dataset will be available in varying portions.

Then using the different portions of available data, fit the models that were used previously, and examine the results when testing on the appropriate data from the testing samples.

*The dataset from part 1 can be treated as the groundtruth, so you can try and random sample from that dataset to produce the testing samples that can be used in this part.* 

***

One can also evaluate on the effect of incorrectness of data discovery.

Suppose what you have done for data discovery is correct, this means the relations that are found are correct.

What if you perform a perturbation on the relations between the files? 

This would mean the dataset will be incorrectly joined, do you think there will be an impact on the outcome of the model that will be trained and then tested on the groudtruth dataset?

What would the effect be on the downstream tasks as mentioned in the machine learning workflow such as data pre-processing, machine learning model training and testing?

This can be evaluated in the same way as above.

In [ ]:
## Do some testing on performing various degrees of incorrect data discovery.
##

## Discussions

1)  Discuss the different effects of the results of the data discovery results on various downstream tasks in the machine learning workflow.
    As stated previously, this is effecting the data loading portion of the ML workflow.

2)  Discuss also what aspects need to be considered when performing data discovery and evaluating the results of data discovery.

Max. 400 words

The perturbations that performed for part 1 and 2 are all data quality issues.

3) Discuss on the effects of data quality and how you may attempt to identify and solve these issues?

<!-- For the set of considerations that you have outlined for the choice of data discovery methods, choose one and identify under this new constraint, how would you identify and resolve this problem? -->

Max 400 words